Workshop URL - [Introduction to Decorators: Power UP Your Python Code](https://www.youtube.com/watch?v=VWZAh1QrqRE)

GitHub Repo - 

In [3]:
def prefix_factory(prefix):
    def prefix_printer(text):
        print(f'{prefix}: {text}')
    return prefix_printer

debug = prefix_factory('DEBUG')
debug('Hi Pycon!')

DEBUG: Hi Pycon!


In [4]:
def reverse_factory(func):
    def reverse_caller(text):
        func(f'{text[::-1]}')
    return reverse_caller

reverse_print = reverse_factory(print)
reverse_print('Hi, Conf')

fnoC ,iH


In [1]:
import sys
sys.version

'3.8.4 (tags/v3.8.4:dfa645a, Jul 13 2020, 16:30:28) [MSC v.1926 32 bit (Intel)]'

In [13]:
@reverse_factory
def greet(text):
    print(f'HI {text}')

greet('Ruby Conf')

HI fnoC ybuR


## Exercise 01

write a decorator thar prints `BEFORE` before calling the decorated function and `AFTER` afterwards.

```py
@before_and_after
def greet(name):
    print(f'Hi {name}')

greet('PyCon')
```

BEFORE <br>
Hi PyCon! <br>
AFTER <br>

In [35]:
def before_and_after(func):
    def wrapper(*args, **kwargs):
        print('BEFORE')
        func(*args, **kwargs)
        print('AFTER')
    return wrapper

@before_and_after
def greet(name):
    print(f'Hi {name}!')

greet('PyCon')

BEFORE
Hi PyCon!
AFTER


In [36]:
def before_and_after_(func):
    def wrapper(*args, **kwargs):
        print('BEFORE')
        print(func(*args, **kwargs))
        print('AFTER')
    return wrapper


@before_and_after_
def adder(a, b):
    return sum((a,b))

adder(1, 10)

BEFORE
11
AFTER


## Exercise 02

write a decorator that runs the decorated function twice and returns a 2-tuple with both return values

```py
import random
@do_twice
def roll_dice():
    return random.randint(1, 6)
```

In [5]:
import random

def do_twice(func):
    def wrapper(*args, **kwargs):
        first_value = func(*args, **kwargs)
        second_value = func(*args, **kwargs)
        
        return first_value, second_value
    return wrapper

@do_twice
def roll_dice():
    return random.randint(1, 6)

roll_dice()

(1, 3)

In [8]:
import random

def define(func):
    print(f'Defining {func.__name__}')
    return func

@define
def roll_dice():
    return random.randint(1, 6)

print(roll_dice())

Defining roll_dice
4


In [9]:
print(roll_dice)

<function roll_dice at 0x7f109453f378>


## Exercise 03

write a decorator that stores references to decorated functions in a dictionary

```py
FUNCTIONS = {}
@register
def roll_dice():
    return random.randint(1, 6)
```

In [25]:
# Mine solution was

import random

FUNCTIONS = {}
def register(func):
    def wrapper(*args, **kwargs):
        FUNCTIONS[func.__name__] = func
        return func(*args, **kwargs)
    return wrapper

@register
def roll_dice():
    return random.randint(1, 6)

roll_dice()
print(FUNCTIONS)
print(FUNCTIONS['roll_dice']())

{'roll_dice': <function roll_dice at 0x7f109453f6a8>}
6


In [26]:
# solution

import random

FUNCTIONS = {}
def register(func):
    FUNCTIONS[func.__name__] = func
    return func

@register
def roll_dice():
    return random.randint(1, 6)

# roll_dice()
print(FUNCTIONS)
print(FUNCTIONS['roll_dice']())

{'roll_dice': <function roll_dice at 0x7f109445d840>}
2


In [8]:
# Does Order Matter? Yes, It's matter

def do_twice(func):
    def wrapper(*args, **kwargs):
        first_value = func(*args, **kwargs)
        second_value = func(*args, **kwargs)
        return first_value, second_value
    return wrapper

def before_and_after(func):
    def wrapper(*args, **kwargs):
        print('BEFORE')
        func(*args, **kwargs)
        print('AFTER')
    return wrapper

@do_twice
@before_and_after
def greet(name):
    print(f'Hi {name}')

# greet = do_twice(before_and_after(greet))
print(greet('PyConf'))

BEFORE
Hi PyConf
AFTER
BEFORE
Hi PyConf
AFTER
(None, None)


In [9]:
def do_twice(func):
    def wrapper(*args, **kwargs):
        first_value = func(*args, **kwargs)
        second_value = func(*args, **kwargs)
        return first_value, second_value
    return wrapper

def before_and_after(func):
    def wrapper(*args, **kwargs):
        print('BEFORE')
        func(*args, **kwargs)
        print('AFTER')
    return wrapper

@before_and_after
@do_twice
def greet(name):
    print(f'Hi {name}')

print(greet('PyConf'))

BEFORE
Hi PyConf
Hi PyConf
AFTER
None


## Exercises 04

write a decorator that repeatedly calls the decorated function as long as it raises an exception

```py
@retry
def only_roll_sixes():
    number = random.randint(1, 6)
    if number != 6:
        raise ValueError(number)
    return number
```

In [15]:
import random

def retry(func):
    def wrapper(*args, **kwargs):
        while True:
            try:
                return func(*args, **kwargs)
            except ValueError as e:
                print(f'Retrying({e})')
    return wrapper

@retry
def only_roll_sixes():
    number = random.randint(1, 6)
    if number != 6:
        raise ValueError(number)
    return number

print(only_roll_sixes())

Retrying(3)
Retrying(4)
Retrying(2)
Retrying(5)
Retrying(3)
Retrying(1)
Retrying(4)
6


## Exercise 05 (Hard)

Rewrite @retry so that it only retries on specific, user-defined exceptions..

```py
@retry(ValueError)
def calculation():
    number = random.randint(-5, 5)
    if abs(1 / number) > 0.2:
        raise ValueError(number)
    return number
```

In [8]:
# we need one more level nesting to solve this problem

import functools
import random

def retry(exception):
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            while True:
                try:
                    return func(*args, **kwargs)
                except exception as e:
                    print(f'Retrying {e}')
        return wrapper
    return deco

@retry(ValueError)
def calculation():
    number = random.randint(-5, 5)
    if abs(1 / number) > 0.2:
        raise ValueError(number)
    return number

print(calculation())

Retrying -2
Retrying -2
Retrying 3
-5


In [ ]:
## Exercise 06

Adapt @retry so that it only tries a maximum of max_retries times

```py
@retry(max_retries=3)
def only_roll_sixes():
    number = random.randint(1, 6)
    if number != 6:
        raise ValueError(number)
    return number
```

In [11]:
import functools
import random

def retry(max_retries=1):
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            for i in range(0, max_retries):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f'Retrying {e}')
        return wrapper
    return deco

@retry(max_retries=3)
def only_roll_sixes():
    number = random.randint(1, 6)
    if number != 6:
        raise ValueError(number)
    return number

print(only_roll_sixes())

Retrying 4
Retrying 3
Retrying 2
None


In [10]:
# we can use class decorator as well

class BeforeAndAfter:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
    def __call__(self, *args, **kwargs):
        print('Before')
        value = self.func(*args, **kwargs)
        print('After')
        return value

@BeforeAndAfter
def greet(name):
    print(f'Hi, {name}')

greet('PyConf')


Before
Hi, PyConf
After


## Exercise 07

write a class based @Retry decorator that keeps track of the number of retries across all function calls

```py
@Retry
def only_roll_sixes():
    number = random.randint(1, 6)
    if number != 6:
        raise ValueError(number)
    return number
```

In [13]:
import random
import functools

class Retry:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_retries = 0
    
    def __call__(self, *args, **kwargs):
        while True:
            try:
                return self.func(*args, **kwargs)
            except Exception:
                self.num_retries += 1
                print(f'Retry attempt {self.num_retries}')

@Retry
def only_roll_sixes():
    number = random.randint(1, 6)
    if number != 6:
        raise ValueError(number)
    return number

print(only_roll_sixes.num_retries)
print(only_roll_sixes())
print(only_roll_sixes.num_retries)

0
Retry attempt 1
Retry attempt 2
Retry attempt 3
Retry attempt 4
Retry attempt 5
Retry attempt 6
Retry attempt 7
6
7
